## Factor Based Trading Strategies
Author: James Hong

### 1. Momentum Factor Construction:

#### Compute a momentum variable for each stock, defined as the cumulative 12-month return from t−11 to t, provided at least 10 months of data are available. This variable is then merged with the main dataset for further analysis.

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from scipy import stats
import statsmodels.api as sm
import math

In [2]:
ca = pd.read_sas('CA.sas7bdat')
ca_drop = ['CUSIP', 'COMNAM', 'TICKER', 'PRC', 'SHROUT','me','gvkey','DATADATE','ATQ','CEQQ','EPSPXQ','IBQ','SALEQ']
ca = ca.drop(ca_drop, axis=1)
print(ca.columns)
ca.head()

Index(['permno', 'date', 'yyyymm', 'ret', 'ret_t1', 'lnsize', 'bk2mkt', 'ep',
       'beta', 'ivol', 'monthid'],
      dtype='object')


,permno,date,yyyymm,ret,ret_t1,lnsize,bk2mkt,ep,beta,ivol,monthid
0,10107.0,2000-01-31,200001.0,-0.161670,-0.086845,13.132388,0.060114,0.004338,1.431645,0.023474,1.0
1,10107.0,2000-02-29,200002.0,-0.086845,0.188811,13.041538,0.065832,0.004751,1.403373,0.017771,2.0
2,10107.0,2000-03-31,200003.0,0.188811,-0.343529,13.230253,0.054510,0.003934,1.429994,0.031297,3.0
3,10107.0,2000-04-28,200004.0,-0.343529,-0.103047,12.813261,0.094234,0.006637,1.599818,0.037296,4.0
4,10107.0,2000-05-31,200005.0,-0.103047,0.278721,12.704509,0.105061,0.007399,1.680414,0.016864,5.0


In [3]:
factors_month = pd.read_sas('factors_monthly.sas7bdat')
factors_month = factors_month[(factors_month['date'] >= '2000-01-01') & (factors_month['date'] <= '2024-11-30')]
factors_month["monthid"] = (factors_month.dateff.dt.year-2000)*12 + factors_month.dateff.dt.month
print(factors_month.columns)
factors_month.head()

Index(['date', 'mktrf', 'smb', 'hml', 'rf', 'year', 'month', 'umd', 'dateff',
       'monthid'],
      dtype='object')


,date,mktrf,smb,hml,rf,year,month,umd,dateff,monthid
882,2000-01-01,-0.0474,0.0577,-0.0188,0.0041,2000.0,1.0,0.0192,2000-01-31,1
883,2000-02-01,0.0245,0.2136,-0.0959,0.0043,2000.0,2.0,0.1820,2000-02-29,2
884,2000-03-01,0.0520,-0.1720,0.0813,0.0047,2000.0,3.0,-0.0683,2000-03-31,3
885,2000-04-01,-0.0640,-0.0668,0.0726,0.0046,2000.0,4.0,-0.0839,2000-04-28,4
886,2000-05-01,-0.0442,-0.0605,0.0475,0.0050,2000.0,5.0,-0.0898,2000-05-31,5


In [4]:
returndata = pd.read_sas('returndata.sas7bdat')
print(returndata.columns)
returndata.head()

Index(['PERMNO', 'DATE', 'RET'], dtype='object')


,PERMNO,DATE,RET
0,10107.0,1997-01-31,0.234493
1,10107.0,1997-02-28,-0.044118
2,10107.0,1997-03-31,-0.059615
3,10107.0,1997-04-30,0.325153
4,10107.0,1997-05-30,0.020576


In [5]:
# Sort the data by PERMNO and DATE
returndata = returndata.sort_values(by=['PERMNO', 'DATE'])
returndata["monthid"] = (returndata.DATE.dt.year-2000)*12 + returndata.DATE.dt.month
# Calculate the cumulative 12-month returns
returndata['mom'] = returndata.groupby('PERMNO')['RET'].rolling(window=12, min_periods=10).apply(lambda x: np.prod(1 + x) - 1).reset_index(level=0, drop=True)
returndata.head(20)

,PERMNO,DATE,RET,monthid,mom
0,10107.0,1997-01-31,0.234493,-35,NaN
1,10107.0,1997-02-28,-0.044118,-34,NaN
2,10107.0,1997-03-31,-0.059615,-33,NaN
3,10107.0,1997-04-30,0.325153,-32,NaN
4,10107.0,1997-05-30,0.020576,-31,NaN
5,10107.0,1997-06-30,0.019153,-30,NaN
6,10107.0,1997-07-31,0.119683,-29,NaN
7,10107.0,1997-08-29,-0.065813,-28,NaN
8,10107.0,1997-09-30,0.000946,-27,NaN
9,10107.0,1997-10-31,-0.017478,-26,0.573374


In [25]:
# Filter returndata for the period of January 2000 to November 2024
filtered_returndata = returndata[(returndata['DATE'] >= '2000-01-01') & (returndata['DATE'] <= '2024-11-30')]

# Merge the filtered returndata with ca dataframe on permno
merged_data = pd.merge(ca, filtered_returndata[['PERMNO', 'DATE', 'mom']], left_on=['permno', 'date'], right_on=['PERMNO', 'DATE'], how='left')

# Drop the redundant columns
merged_data = merged_data.drop(columns=['PERMNO', 'DATE'])

# Report the summary statistics of "mom" in the merged data
summary_stats = merged_data['mom'].describe(percentiles=[0.01, 0.5, 0.99])
summary_stats.loc['N'] = merged_data['mom'].count()
summary_stats = summary_stats.rename({'50%': 'median', '1%': '1st percentile', '99%': '99th percentile'})

print(summary_stats[['N', 'std', 'median', 'min', '1st percentile', '99th percentile', 'max']].round(4))

N                  22467.0000
std                    0.8235
median                 0.1692
min                   -0.9723
1st percentile        -0.6395
99th percentile        2.5114
max                   53.6628
Name: mom, dtype: float64


In [7]:
merged_data.head()

,permno,date,yyyymm,ret,ret_t1,lnsize,bk2mkt,ep,beta,ivol,monthid,mom
0,10107.0,2000-01-31,200001.0,-0.161670,-0.086845,13.132388,0.060114,0.004338,1.431645,0.023474,1.0,0.118571
1,10107.0,2000-02-29,200002.0,-0.086845,0.188811,13.041538,0.065832,0.004751,1.403373,0.017771,2.0,0.190674
2,10107.0,2000-03-31,200003.0,0.188811,-0.343529,13.230253,0.054510,0.003934,1.429994,0.031297,3.0,0.185495
3,10107.0,2000-04-28,200004.0,-0.343529,-0.103047,12.813261,0.094234,0.006637,1.599818,0.037296,4.0,-0.142198
4,10107.0,2000-05-31,200005.0,-0.103047,0.278721,12.704509,0.105061,0.007399,1.680414,0.016864,5.0,-0.224632


### 2. Outlier Handling (Winsorization):
#### To mitigate the influence of extreme values, selected financial and risk-related variables—including firm size (lnSize), book-to-market (bk2mkt), earnings-price ratio (ep), beta, idiosyncratic volatility (ivol), and momentum—are winsorized at the 1st and 99th percentile levels on a monthly basis.

In [27]:
def winsorize_series(series, lower_quantile=0.01, upper_quantile=0.99):
    lower_bound = series.quantile(lower_quantile)
    upper_bound = series.quantile(upper_quantile)
    return series.clip(lower=lower_bound, upper=upper_bound)

columns_to_winsorize = ['lnsize', 'bk2mkt', 'ep', 'beta', 'ivol', 'mom']

# Add the winsorized values to the dataset with a new variable name
for column in columns_to_winsorize:
    merged_data[f'{column}_winsorized'] = merged_data.groupby(merged_data['date'].dt.to_period('M'))[column].transform(winsorize_series)

# Compare the standard deviation of raw and winsorized variables
std_comparison = pd.DataFrame({
    'Variable': columns_to_winsorize,
    'Raw Std': merged_data[columns_to_winsorize].std().values,
    'Winsorized Std': merged_data[[f'{col}_winsorized' for col in columns_to_winsorize]].std().values
}).set_index('Variable')

std_comparison.head()

,Raw Std,Winsorized Std
Variable,,
lnsize,1.675192,1.655453
bk2mkt,0.377279,0.300819
ep,0.043886,0.028563
beta,0.799977,0.764341
ivol,0.011511,0.010943


In [9]:
# Drop the raw columns since we have their winsorized versions
columns_to_drop = ['lnsize', 'bk2mkt', 'ep', 'beta', 'ivol', 'mom']
merged_data = merged_data.drop(columns=columns_to_drop)

# Rename the winsorized columns back to original names
merged_data = merged_data.rename(columns={
    'lnsize_winsorized': 'lnsize',
    'bk2mkt_winsorized': 'bk2mkt',
    'ep_winsorized': 'ep',
    'beta_winsorized': 'beta',
    'ivol_winsorized': 'ivol',
    'mom_winsorized': 'mom'
})

# Verify the columns after dropping
print("Columns after dropping raw variables:")
print(merged_data.columns)

Columns after dropping raw variables:
Index(['permno', 'date', 'yyyymm', 'ret', 'ret_t1', 'monthid', 'lnsize',
       'bk2mkt', 'ep', 'beta', 'ivol', 'mom'],
      dtype='object')


### 3.Portfolio Construction and Performance Evaluation:
#### Using a [m,n,l] month rule (with l=1 and n=0), stocks are sorted into quintile portfolios based on the six key factors. Equal-weighted portfolios are constructed monthly, and hedge portfolios (Q5−Q1) are formed. These are evaluated based on return metrics, Sharpe ratio, CAPM and Fama-French factor alphas.

In [10]:
def make_quintile(factor, data):
    # beta rank process (10 quantiles hence the use of quantile cut)
    data.sort_values(["monthid", f"{factor}"])

    data[f"{factor}_rank"] = data.groupby(['monthid'])[f'{factor}'].transform(lambda x: pd.qcut(x, 5, labels=False))
    data.sort_values(["monthid", f"{factor}_rank"])
    # grouping by beta ranking (decile) 
    data[f"monthid_{factor}"] = data["monthid"].astype(str) + data[f"{factor}_rank"].astype(str)
    factorGroups = data.groupby([f"monthid_{factor}"])

    quintile = {'monthid':[], f'{factor}_rank':[], 'freq':[], 'return':[], f'{factor}':[]}#, 'ff3_beta':[], 'smb_beta':[], 'hml_beta':[]}

    #for each decile calculate the average return, beta, etc
    for name, group in factorGroups:
        quintile['monthid'].append(group['monthid'].iloc[0])
        quintile[f'{factor}_rank'].append(group[f'{factor}_rank'].iloc[0])
        quintile['freq'].append(group.shape[0])
        quintile['return'].append(group['ret'].mean())
        quintile[f'{factor}'].append(group[f"{factor}"].mean())
        # quintile['ff3_beta'].append(group['ff3_beta'].mean())
        # quintile['smb_beta'].append(group['smb_beta'].mean())
        # quintile['hml_beta'].append(group['hml_beta'].mean())

    quintileDF = pd.DataFrame(quintile)
    quintileDF.sort_values(["monthid", f'{factor}_rank'])
    quintileDF = quintileDF.dropna().reset_index(drop=True)
    return quintileDF

In [11]:
merged_data.head(20)

,permno,date,yyyymm,ret,ret_t1,monthid,lnsize,bk2mkt,ep,beta,ivol,mom
0,10107.0,2000-01-31,200001.0,-0.161670,-0.086845,1.0,12.943430,0.060114,0.004338,1.431645,0.023474,0.118571
1,10107.0,2000-02-29,200002.0,-0.086845,0.188811,2.0,13.006111,0.065832,0.004751,1.403373,0.017771,0.190674
2,10107.0,2000-03-31,200003.0,0.188811,-0.343529,3.0,13.177262,0.054510,0.003934,1.429994,0.031297,0.185495
3,10107.0,2000-04-28,200004.0,-0.343529,-0.103047,4.0,12.813261,0.094234,0.006637,1.599818,0.037296,-0.142198
4,10107.0,2000-05-31,200005.0,-0.103047,0.278721,5.0,12.704509,0.105061,0.007399,1.680414,0.016864,-0.224632
5,10107.0,2000-06-30,200006.0,0.278721,-0.127344,6.0,12.950370,0.082161,0.005786,1.767042,0.021499,-0.112959
6,10107.0,2000-07-31,200007.0,-0.127344,0.000000,7.0,12.814157,0.108312,0.006492,1.820740,0.020498,-0.186453
7,10107.0,2000-08-31,200008.0,0.000000,-0.136079,8.0,12.814157,0.108312,0.006492,1.732500,0.015628,-0.245780
8,10107.0,2000-09-29,200009.0,-0.136079,0.141969,9.0,12.681084,0.123729,0.007416,1.813573,0.015832,-0.334023
9,10107.0,2000-10-31,200010.0,0.141969,-0.166969,10.0,12.813838,0.112638,0.006559,1.762680,0.047211,-0.255908


In [12]:
# Create a list of factors to analyze
factors = ['lnsize', 'bk2mkt', 'ep', 'beta', 'ivol', 'mom']
# Create an empty list to store the DataFrames
quintile_dfs = []

# Apply make_quintile() to each factor and store the results
for factor in factors:
    df = make_quintile(factor, merged_data)
    df.name = factor  # Add name attribute to identify the DataFrame
    quintile_dfs.append(df)

In [13]:
quintile_dfs[5]

,monthid,mom_rank,freq,return,mom
0,1.0,0.0,12,-0.118153,-0.302982
1,1.0,1.0,11,-0.055438,0.003530
2,1.0,2.0,11,0.075278,0.267215
3,1.0,3.0,11,-0.020425,0.872141
4,1.0,4.0,11,0.016120,3.292423
...,...,...,...,...,...
1490,99.0,0.0,13,-0.069768,-0.396593
1491,99.0,1.0,13,0.041421,-0.164585
1492,99.0,2.0,13,0.008948,-0.043970
1493,99.0,3.0,13,0.014684,0.107940


In [28]:
# Create Excel writer object
with pd.ExcelWriter('quintile_portfolios.xlsx') as writer:
    # Loop through each DataFrame in quintile_dfs and save to a separate sheet
    for df, factor in zip(quintile_dfs, factors):
        # Save each DataFrame to a sheet named after the factor
        df.to_excel(writer, sheet_name=factor, index=False)

In [14]:
def hedge_portfolio(factor, quintile_dfs):
    factor_index = factors.index(factor)

    firstQuintile = quintile_dfs[factor_index].loc[quintile_dfs[factor_index][f'{factor}_rank']==0]
    lastQuintile = quintile_dfs[factor_index].loc[quintile_dfs[factor_index][f'{factor}_rank']==4]
    hedge = pd.merge(firstQuintile, lastQuintile, on=["monthid"])
    if factor == 'mom' or factor == 'bk2mkt' or factor == 'ep':
        hedge['hedge_ret'] = hedge['return_y'] - hedge['return_x']
    elif factor == 'lnsize' or factor == 'ivol' or factor == 'beta':
        hedge['hedge_ret'] = hedge['return_x'] - hedge['return_y']
    return hedge

In [16]:
# Store hedge portfolios for each factor
hedge_portfolios = []

# Loop through each factor and create hedge portfolios
for factor in factors:
    hedge_ff4 = hedge_portfolio(factor, quintile_dfs)
    hedge_ff4 = pd.merge(hedge_ff4, factors_month, on=["monthid"])
    hedge_ff4.sort_values(["monthid"],inplace=True)
    hedge_portfolios.append(hedge_ff4)
hedge_portfolios[1]


,monthid,bk2mkt_rank_x,freq_x,return_x,bk2mkt_x,bk2mkt_rank_y,freq_y,return_y,bk2mkt_y,hedge_ret,date,mktrf,smb,hml,rf,year,month,umd,dateff
0,1.0,0.0,11,0.027099,0.041217,4.0,11,-0.071195,0.622311,-0.098294,2000-01-01,-0.0474,0.0577,-0.0188,0.0041,2000.0,1.0,0.0192,2000-01-31
111,2.0,0.0,11,0.267784,0.035751,4.0,11,-0.037279,0.638013,-0.305064,2000-02-01,0.0245,0.2136,-0.0959,0.0043,2000.0,2.0,0.1820,2000-02-29
222,3.0,0.0,11,0.044650,0.033825,4.0,11,0.023097,0.610373,-0.021554,2000-03-01,0.0520,-0.1720,0.0813,0.0047,2000.0,3.0,-0.0683,2000-03-31
233,4.0,0.0,12,-0.081539,0.034108,4.0,11,-0.023315,0.777657,0.058224,2000-04-01,-0.0640,-0.0668,0.0726,0.0046,2000.0,4.0,-0.0839,2000-04-28
244,5.0,0.0,12,-0.063870,0.040241,4.0,11,-0.040632,0.826087,0.023239,2000-05-01,-0.0442,-0.0605,0.0475,0.0050,2000.0,5.0,-0.0898,2000-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,295.0,0.0,20,-0.033603,0.019186,4.0,20,0.037189,0.541136,0.070792,2024-07-01,0.0124,0.0680,0.0574,0.0045,2024.0,7.0,-0.0242,2024-07-31
218,296.0,0.0,20,0.090474,0.018611,4.0,20,-0.040609,0.563606,-0.131084,2024-08-01,0.0161,-0.0355,-0.0113,0.0048,2024.0,8.0,0.0479,2024-08-30
219,297.0,0.0,20,0.023506,0.018745,4.0,20,0.023312,0.546461,-0.000194,2024-09-01,0.0174,-0.0017,-0.0259,0.0040,2024.0,9.0,-0.0060,2024-09-30
220,298.0,0.0,20,0.052631,0.018810,4.0,20,-0.032810,0.586056,-0.085442,2024-10-01,-0.0097,-0.0101,0.0089,0.0039,2024.0,10.0,0.0287,2024-10-31


In [19]:
def hedge_regression_result(hedge_ff4):

    hedge_stat = {'ret':[], 'xret':[],'sharpe' : [],
                  'capm_alpha':[], 'capm_alpha_tvalue':[], 
                  'ff4_alpha':[], 'ff4_alpha_tvalue':[]}
    
    hedge_ff4["x_hedge_ret"] = hedge_ff4["hedge_ret"] - hedge_ff4["rf"]
 
    # CAPM regression
    CAPMmodel = sm.OLS(hedge_ff4["x_hedge_ret"], sm.add_constant(hedge_ff4["mktrf"])).fit()
    CAPMalpha = CAPMmodel.params['const']
    CAPMalpha_pvalue = CAPMmodel.tvalues.iloc[0]

    # FF3 regression
    FF4model = sm.OLS(hedge_ff4["x_hedge_ret"], sm.add_constant(hedge_ff4[["mktrf", "smb", "hml", "umd"]])).fit()
    # FF4model = linear_model.LinearRegression().fit(hedge_ff4[["mktrf", "smb", "hml", "umd"]], hedge_ff4["hedge_ret"])
    FF4alpha = FF4model.params['const']
    FF4alpha_pvalue = FF4model.tvalues['const']
    #FF4alpha_pvalue = sm.OLS(hedge_ff4["hedge_ret"], sm.add_constant(hedge_ff4[["mktrf", "smb", "hml", "umd"]])).fit().pvalues.iloc[0]

    hedge_stat['capm_alpha'].append(round(CAPMalpha, 4))
    hedge_stat['capm_alpha_tvalue'].append(round(CAPMalpha_pvalue,4))
    hedge_stat['ff4_alpha'].append(round(FF4alpha,4))
    hedge_stat['ff4_alpha_tvalue'].append(round(FF4alpha_pvalue,4))
    hedge_stat['sharpe'].append(round(12*(hedge_ff4['hedge_ret'].mean()/(math.sqrt(12)*hedge_ff4['hedge_ret'].std())),4))
    hedge_stat['ret'].append(round(hedge_ff4['hedge_ret'].mean(),4))
    hedge_stat['xret'].append(round(hedge_ff4['hedge_ret'].mean() - hedge_ff4['mktrf'].mean() - hedge_ff4["rf"].mean(),4))
    hedge_stat_df = pd.DataFrame(hedge_stat)

    return hedge_stat_df

In [20]:
# create hedge_stat_df for each factor
hedge_stat_dfs = []
for hedge_ff4, factor in zip(hedge_portfolios, factors):
    # Drop NaN values before performing regression
    hedge_stat_df = hedge_regression_result(hedge_ff4)
    hedge_stat_df.index = [factor]  # Set the index to the factor name
    hedge_stat_dfs.append(hedge_stat_df)

# Concatenate all DataFrames vertically
hedge_stat_dfs = pd.concat(hedge_stat_dfs)

In [22]:
hedge_stat_dfs

,ret,xret,sharpe,capm_alpha,capm_alpha_tvalue,ff4_alpha,ff4_alpha_tvalue
lnsize,0.0062,-0.0013,0.3677,0.0023,0.7128,0.0024,0.8796
bk2mkt,-0.0319,-0.0394,-1.8226,-0.0307,-9.1760,-0.0316,-11.4853
ep,-0.0230,-0.0305,-1.0491,-0.0212,-5.0588,-0.0226,-6.7458
beta,-0.0095,-0.0170,-0.3138,-0.0008,-0.1920,-0.0038,-1.1174
ivol,-0.0242,-0.0317,-0.8592,-0.0192,-3.9006,-0.0224,-5.9000
mom,0.0679,0.0604,2.7509,0.0688,14.1050,0.0635,16.7351


### 4. Factor-Mimicking ETF Simulation:
#### Inspired by real-world low-volatility ETFs like BlackRock’s USMV, a long-only ETF is constructed using the long leg of the idiosyncratic volatility-based hedge strategy. Performance metrics (excess return and alpha) and annual portfolio turnover are computed to simulate realistic investment conditions, including considerations for transaction costs.

In [ ]:
#hedge port 5 contains ivol port:

last_quintile_ivol = quintile_dfs[4][quintile_dfs[4]['ivol_rank'] == 4]
last_quintile_ivol = pd.merge(last_quintile_ivol,factors_month,on=['monthid'])

X = last_quintile_ivol["mktrf"].values.reshape(-1, 1)
y = last_quintile_ivol["return"].values - last_quintile_ivol.rf
X_const = sm.add_constant(X)
# Fit the OLS model
model = sm.OLS(y, X_const).fit()
# Extract the intercept and its t-statistic
alpha = model.params["const"]
alpha_tvalue = model.tvalues["const"]

# for FF 4 model:
ff3explanatory = sm.add_constant(last_quintile_ivol[["mktrf", "smb", "hml",'umd']])
ff3explanatory.reset_index(drop=True, inplace=True)
# dependent is excess return at 2019 dec

# run the linear models again
ff3_model = sm.OLS(y, ff3explanatory).fit()
ff_alpha = ff3_model.params["const"]
t_stat_ff_alpha = ff3_model.tvalues["const"]

print(f"The fama-french four-factor alpha is {ff_alpha}")
print(f"The t-value of fama-french four-factor alpha is {t_stat_ff_alpha}")
# Print the excess returns over market returns
excess_return = last_quintile_ivol["return"].mean() - last_quintile_ivol["mktrf"].mean()
print(f"The mean excess return over market return is: {excess_return:.4%}")

0.02415507540295145
5.276670014896658
The fama-french four-factor alpha is 0.02699238274795218
The t-value of fama-french four-factor alpha is 7.485488211579958
The mean excess return over market return is: 3.0683%


In [52]:
# Create a DataFrame for the ETF
ETF = merged_data.copy()

# Create a dummy variable for stocks in the highest ivol quintile (in_t)
ETF['in_t'] = (ETF.ivol >= ETF.groupby('monthid')['ivol'].transform(lambda x: x.quantile(0.8)))

# Sort by permno to ensure proper calculation of lagged values
ETF = ETF.sort_values(['permno', 'monthid'])

# Create lagged in_t variable (in_t_minus_1)
ETF['in_t_minus_1'] = ETF.groupby('permno')['in_t'].shift(1)

# Identify new additions to the portfolio
ETF['new'] = (ETF.in_t == True) & (ETF.in_t_minus_1 == False)

# Calculate monthly turnover
monthly_turnover = []

for name, group in ETF.groupby('monthid'):
    total = len(group[group.in_t == True]) + 1  # Total positions in portfolio
    new_positions = len(group[group.new == True])  # New positions added
    monthly_turnover.append(new_positions / total)

# Calculate average monthly and annual turnover
mean_turnover = np.mean(monthly_turnover)
annual_turnover = 12 * mean_turnover

print(f'The annual turnover (in %) of the ETF is: {round(annual_turnover*100, 2)}')

The annual turnover (in %) of the ETF is: 566.82


### 5. Fama-MacBeth Cross-Sectional Regressions:
#### A monthly cross-sectional regression is conducted to test the predictive power of idiosyncratic volatility on future returns, controlling for size, beta, and book-to-market. This step assesses the significance of idiosyncratic volatility in expected return models.

In [34]:
q5_merged= pd.merge(merged_data, factors_month, on=["monthid"])
q5_merged


,permno,date_x,yyyymm,ret,ret_t1,lnsize,bk2mkt,ep,beta,ivol,...,mom_winsorized,date_y,mktrf,smb,hml,rf,year,month,umd,dateff
0,10107.0,2000-01-31,200001.0,-0.161670,-0.086845,13.132388,0.060114,0.004338,1.431645,0.023474,...,0.118571,2000-01-01,-0.0474,0.0577,-0.0188,0.0041,2000.0,1.0,0.0192,2000-01-31
1,10145.0,2000-01-31,200001.0,-0.167931,0.006510,10.542263,0.143151,0.014624,1.085556,0.021703,...,0.246396,2000-01-01,-0.0474,0.0577,-0.0188,0.0041,2000.0,1.0,0.0192,2000-01-31
2,11403.0,2000-01-31,200001.0,-0.140625,-0.033333,8.521650,0.195910,-0.008252,1.630000,0.042632,...,-0.355469,2000-01-01,-0.0474,0.0577,-0.0188,0.0041,2000.0,1.0,0.0192,2000-01-31
3,11618.0,2000-01-31,200001.0,-0.026426,0.011829,7.414471,0.160249,0.010296,1.352955,0.043833,...,0.135716,2000-01-01,-0.0474,0.0577,-0.0188,0.0041,2000.0,1.0,0.0192,2000-01-31
4,13856.0,2000-01-31,200001.0,-0.031915,-0.058608,10.813271,0.138539,0.009743,1.299417,0.022216,...,-0.111746,2000-01-01,-0.0474,0.0577,-0.0188,0.0041,2000.0,1.0,0.0192,2000-01-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22942,92221.0,2024-11-29,202411.0,-0.025530,-0.143429,11.519327,0.036327,0.005276,1.592319,0.037473,...,0.225066,2024-11-01,0.0651,0.0463,-0.0005,0.0040,2024.0,11.0,0.0090,2024-11-29
22943,93002.0,2024-11-29,202411.0,-0.045297,0.434045,13.540425,0.086439,-0.001885,1.222581,0.008709,...,0.777256,2024-11-01,0.0651,0.0463,-0.0005,0.0040,2024.0,11.0,0.0090,2024-11-29
22944,93089.0,2024-11-29,202411.0,0.070945,-0.062506,10.634549,0.010357,0.007416,0.867076,0.006901,...,0.225918,2024-11-01,0.0651,0.0463,-0.0005,0.0040,2024.0,11.0,0.0090,2024-11-29
22945,93132.0,2024-11-29,202411.0,0.208365,-0.005997,11.196176,0.003956,0.005213,0.986179,0.030033,...,0.808409,2024-11-01,0.0651,0.0463,-0.0005,0.0040,2024.0,11.0,0.0090,2024-11-29


In [ ]:
dependet = q5_merged["ret"]-q5_merged["rf"]-q5_merged["mktrf"]

# for FF 4 model:
ff3explanatory = sm.add_constant(q5_merged[['lnsize', 'bk2mkt', 'beta', 'ivol']])
ff3explanatory.reset_index(drop=True, inplace=True)
# dependent is excess return at 2019 dec

# run the linear models again
ff3_model = sm.OLS(dependet, ff3explanatory).fit()
ff_alpha = ff3_model.params["const"]
t_stat_ff_alpha = ff3_model.tvalues["const"]
ff3_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   xret   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     69.06
Date:                Tue, 18 Mar 2025   Prob (F-statistic):           3.37e-58
Time:                        23:01:56   Log-Likelihood:                 16077.
No. Observations:               22018   AIC:                        -3.214e+04
Df Residuals:                   22013   BIC:                        -3.210e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0108      0.006     -1.779      0.075      -0.023       0.001
lnsize         0.0021      0.001      4.081      0.000       0.001       0.003
bk2mkt        -0.0186      0.002     -8.687      0.000      -0.023      -0.014
beta          -0.0019      0.001     -1.787      0.074      -0.004       0.000
ivol           1.0157      0.078     12.972      0.000       0.862       1.169
==============================================================================
Omnibus:                    12238.062   Durbin-Watson:                   2.050
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1554336.978
Skew:                           1.695   Prob(JB):                         0.00
Kurtosis:                      44.022   Cond. No.                     1.01e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""